# Part 2 - Setting up a mediation for Alice via a mediator

### Initialise the multitenant controller

In [ ]:
%autoawait
import time
import asyncio
import pprint

from aries_basic_controller.aries_controller import AriesAgentController
from aries_basic_controller.aries_tenant_controller import AriesTenantController

# Create a small utility to print json formatted outout more human-readable    
pp = pprint.PrettyPrinter(indent=4)

WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_BASE = ""

WEBHOOK_PORT = 8022
ADMIN_URL = "http://multitenant-agent:8021"


In [ ]:
%store -r wallet_id_alice
%store -r alice_jwt
print(wallet_id_alice)

In [ ]:
# Based on the aca-py agent you wish to control
alice_agent_controller = AriesTenantController(admin_url=ADMIN_URL, wallet_id=wallet_id_alice, tenant_jwt=alice_jwt)

In [ ]:
agent_controller = AriesAgentController(admin_url=ADMIN_URL, api_key="password", is_multitenant=True)

In [ ]:
agent_controller.init_webhook_server(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT, webhook_base=WEBHOOK_BASE)

In [ ]:
await agent_controller.listen_webhooks()

In [ ]:
def connection_handler(payload):
    print("Connection Handler Called")
    connection_id = payload["connection_id"]
    state = payload["state"]
    print(f"Connection {connection_id} in State {state}")
    
connection_listener = {
    "handler": connection_handler,
    "topic": "connections"
}

alice_agent_controller.register_listeners([connection_listener], defaults=True)

### Updating JWT of the agent controller

Retrieve Alice's token we have stored previously

In [ ]:
%store -r alice_jwt

In [ ]:
print(alice_jwt)

Now we can update the agent controller with the JWT Token

In [ ]:
alice_agent_controller.update_tenant_jwt(tenant_jwt=alice_jwt, wallet_id=wallet_id_alice)

Let's check it's really there

In [ ]:
print(alice_agent_controller.tenant_jwt)

### Let's create an invitation

In [ ]:
# Create Invitation
unmediated_invite = await alice_agent_controller.connections.create_invitation()

We have created an invitation now. Don't worry about this until the end of the notebook. Then you should worry. This will be used to demonstrate something relevant later on.

### Go to the [mediation agent](http://localhost:8890/notebooks/Configure%20Mediator.ipynb) before you continue to generate and fetch the invitation



### Accept Invite From Mediator

Replace the invitation object below with the one you have generated in the mediator notebook

In [ ]:
mediator_invitation = {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': '82c4b84e-a76f-4ebc-a024-f24a1bde880f', 'label': 'MEDIATOR', 'recipientKeys': ['DCRBaGyHP6BwiSpqDBnL4h9tR2MNGAVVykifpRUS7e5N'], 'serviceEndpoint': 'http://3615ce378683.ngrok.io'}

In [ ]:
response = await alice_agent_controller.connections.accept_connection(mediator_invitation)
pp.pprint(response)

In [ ]:
connection_id = response["connection_id"]
print(connection_id)

### Request mediation

Now that we have successfully established a connection between Alice and the mediator agent we can proceed to request mediation from the mediator.

In [ ]:
### check state of connection
connection = await alice_agent_controller.connections.get_connection(connection_id)
print(connection['state'])

In [ ]:
# Let's check for the state
if connection['state'] != 'active':
    print("No active connection. \n Please go back and ensure you have established an active connection between the mediator agent and Alice's subwallet agent")    
else:
    ## request mediation
    mediation_req = await alice_agent_controller.mediation.request_mediation(connection_id)
    print(mediation_req)


### Let's have a look at the mediation records and we should see our mediation in there

In [ ]:
response = await alice_agent_controller.mediation.get_mediation_records()
print(response)

### Set a default mediator

By setting a default mediator, all DIDComm connections we establish will be routed through this mediator. To do this we use the appropriate API endpoint via the agent controller and pass it the mediation ID of our mediated connection.

In [ ]:
default_mediation_res = await alice_agent_controller.mediation.set_default_mediator(response[0]['mediation_id'])
pp.pprint(default_mediation_res)

### Check whether our default mediator is really there

In [ ]:
default_mediator = await alice_agent_controller.mediation.get_default_mediator()
pp.pprint(default_mediator)

if default_mediator['connection_id'] != connection_id:
    print("Oooops! Something went wrong setting the default mediator. Please, check above and try again")
else:
    print("\n\n Hooray! We have succesfully set a default mediator.")


In [ ]:
# Create Invitation
invite = await alice_agent_controller.connections.create_invitation()
connection_id = invite["connection_id"]
invite_message = invite['invitation']
print("Connection ID", connection_id)
print("Invitation")
print(invite_message)
pp.pprint("\n" + invite_message['routingKeys'][0])

### Checking routing keys

This routing key should be used from now on to encrypt all messages/comminucation. Below you'll see that going back our initially created invitation we don't have such key.

In [ ]:
unmediated_invite_message = unmediated_invite['invitation']
pp.pprint(unmediated_invite_message)

As you can see, there is no routing key in there

### Comparing endpoints

Let's check and see that the service endpoint of the mediated connection is now actually the one we got from the invitation from the mediator agent.

We'll also see that this is not the endpoint for the unmediated invitation.

In [ ]:
print("Unmediated endpoint: " + unmediated_invite_message['serviceEndpoint'] + "\n\n")
print("Mediated endpoint: " + invite_message['serviceEndpoint'])
print("Mediator Invitation Ednpoint: " + mediator_invitation['serviceEndpoint'])

### Great. You're done with this tutorial. Please move on part 3..

In [ ]:
await agent_controller.terminate()
await alice_agent_controller.terminate()